# SVHN Digit Recognition
## Katherine King

SVHN is a dataset of street view house numbers obtained from Google (http://ufldl.stanford.edu/housenumbers/). Recognizing digits in SVHN is more difficult than in MNIST. 
Reference: Y. Netzer, T. Wang, A. Coates, A. Bissacco, B. Wu and A. Y. Ng. "Reading Digits in Natural Images with Unsupervised Feature Learning". NIPS Workshop on Deep Learning and Unsupervised Feature Learning, 2011.

This notebook contains an end-to-end workflow for building, training, validating, evaluating and saving multiclass MLP and CNN models using the Sequential API in Keras.

In [ ]:
import tensorflow as tf
from scipy.io import loadmat
from sklearn.preprocessing import OneHotEncoder
import random
import matplotlib.pyplot as plt
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam


In [ ]:
# Run this cell to load the dataset

train = loadmat('data/train_32x32.mat')
test = loadmat('data/test_32x32.mat')

# Exploration and pre-processing

1. Extract the training and testing images and labels.
2. Select and display a random sample of images and corresponding labels from the dataset.
3. Convert the training and test images to grayscale by taking the average across all color channels for each pixel, retaining the channel dimension (now size=1). 
4. Select and display a random sample of the grayscale images and corresponding labels.

In [ ]:
x_train = train['X'] / 255.
y_train = train['y']
x_test = test['X'] / 255.
y_test = test['y']
y_train = np.where(y_train==10, 0, y_train)
y_test = np.where(y_test==10, 0, y_test)

In [ ]:
x_train = np.moveaxis(x_train, -1, 0)
x_test = np.moveaxis(x_test, -1 , 0)
print(x_train.shape, x_test.shape)

In [ ]:
indices = random.sample(range(0, x_train.shape[0]), 10 )
fig, ax = plt.subplots(1, 10, figsize=(10,1))

for i in range(10):
    ax[i].set_axis_off()
    ax[i].imshow(x_train[indices[i]])
    ax[i].set_title(y_train[indices[i]])

In [ ]:
x_train_gs = np.mean(x_train, -1, keepdims=True)
x_test_gs = np.mean(x_test, -1, keepdims=True)
x_train_gs.shape

In [ ]:
indices = random.sample(range(0, x_train_gs.shape[0]), 10 )
fig, ax = plt.subplots(1, 10, figsize=(10,1))

for i in range(10):
    ax[i].set_axis_off()
    ax[i].imshow(x_train_gs[indices[i],:,:,0], cmap='gray')
    ax[i].set_title(y_train[indices[i]])

# MLP neural network classifier


In [ ]:
def get_mlp_model():
    model = Sequential([
        Flatten(input_shape=x_train[0].shape),
        Dense(512, activation='relu'),
        Dense(512, activation='relu'),
        Dense(256, activation='relu'),
        Dense(128, activation='relu'),
        Dense(10, activation='softmax')
    ])
    return model
model=get_mlp_model()

In [ ]:
model.summary()

In [ ]:
model.compile(optimization='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
checkpoint_path = 'checkpoint/best_model'
checkpoint = ModelCheckpoint(checkpoint_path,
                     save_best_only=True,
                     save_weights_only=True,
                     verbose=False,
                     save_freq='epoch',
                     monitor='val_loss',
                     mode='min')

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', mode='min', patience=4)
history = model.fit(x_train, y_train, epochs=30, batch_size=64, 
                    verbose=1, validation_split=0.1, callbacks=[checkpoint, early_stop])

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title("Loss Vs Epochs")
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['Training Loss', 'Validation Loss'], loc='upper right')
plt.show()

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title("Accuracy Vs Epochs")
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['Training Accuracy', 'Validation Accuracy'], loc='lower right')
plt.show()

In [ ]:
test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=False)
print(f"Test Loss is {test_loss}")
print(f"Test Accuracy is {test_accuracy}")

# CNN neural network classifier

In [ ]:
def get_cnn_model():
    cnn_model = Sequential([
        Conv2D(64, (3,3), activation='relu', input_shape=(32, 32, 3)),
        MaxPooling2D((3,3)),
        BatchNormalization(), 
        Conv2D(16, (3,3), activation='relu'),
        MaxPooling2D((3,3)),
        BatchNormalization(),
        Dropout(0.5),
        Flatten(),
        Dense(64, activation='relu'),
        Dropout(0.3),
        Dense(10, activation='softmax')
    ])
    return cnn_model  
cnn_model = get_cnn_model()


In [ ]:
cnn_model.summary()

In [ ]:
cnn_model.compile(optimization='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
cnn_checkpoint_path = 'cnn_checkpoint/best_cnn_model'
cnn_checkpoint = ModelCheckpoint(cnn_checkpoint_path,
                     save_best_only=True,
                     save_weights_only=True,
                     verbose=False,
                     save_freq='epoch',
                     monitor='val_loss',
                     mode='min')

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', mode='min', patience=3)
cnn_history = cnn_model.fit(x_train, y_train, epochs=30, batch_size=64, 
                    verbose=1, validation_split=0.1, callbacks=[cnn_checkpoint, early_stop])

In [ ]:
plt.plot(cnn_history.history['loss'])
plt.plot(cnn_history.history['val_loss'])
plt.title("Loss Vs Epochs")
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['Training Loss', 'Validation Loss'], loc='upper right')
plt.show()

In [ ]:
plt.plot(cnn_history.history['accuracy'])
plt.plot(cnn_history.history['val_accuracy'])
plt.title("Accuracy Vs Epochs")
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['Training Accuracy', 'Validation Accuracy'], loc='lower right')
plt.show()

In [ ]:
cnn_test_loss, cnn_test_accuracy = cnn_model.evaluate(x_test, y_test, verbose=False)
print(f"Test Loss is {cnn_test_loss}")
print(f"Test Accuracy is {cnn_test_accuracy}")

# Get model predictions

In [ ]:
best_mlp_model = get_mlp_model()
best_mlp_model.load_weights('checkpoint/best_model')
best_cnn_model = get_cnn_model()
best_cnn_model.load_weights('cnn_checkpoint/best_cnn_model')

In [ ]:
def display_predictive_distribution(model):

    num_test_images = x_test.shape[0]

    random_inx = np.random.choice(num_test_images, 5)
    random_test_images = x_test[random_inx, ...]
    random_test_labels = y_test[random_inx, ...]

    fig, axes = plt.subplots(5, 2, figsize=(16, 12))
    fig.subplots_adjust(hspace=0.4, wspace=-0.2)

    predictions = model.predict(random_test_images)
    
    for i, (prediction, image, label) in enumerate(zip(predictions, random_test_images, random_test_labels)):
        axes[i, 0].imshow(np.squeeze(image))
        axes[i, 0].get_xaxis().set_visible(False)
        axes[i, 0].get_yaxis().set_visible(False)
        axes[i, 0].text(10., -1.5, f'Digit {label}')
        axes[i, 1].bar(np.arange(len(prediction)), prediction)
        axes[i, 1].set_xticks(np.arange(len(prediction)))
        axes[i, 1].set_title(f"Model prediction: {np.argmax(prediction)}")
    
    plt.show()

In [ ]:
display_predictive_distribution(best_mlp_model)

In [ ]:
display_predictive_distribution(best_cnn_model)